In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import nltk
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModel, DistilBertModel


nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load abstracts
abstracts = {}
with open("/content/drive/MyDrive/llms/abstracts.txt", "r", encoding="utf-8") as f:
    content = f.read()
    abstract_entries = content.split("\n")

    for entry in abstract_entries:
        if entry and "|--|" in entry:
            parts = entry.split("|--|")
            if len(parts) == 2:
                paper_id = int(parts[0])
                abstract_text = parts[1]
                abstracts[paper_id] = abstract_text
    print(f"Loaded {len(abstracts)} abstracts.")

Loaded 138499 abstracts.


In [ ]:
# Load authors
authors = {}
with open("/content/drive/MyDrive/llms/authors.txt", "r", encoding="utf-8") as f:
    for line in f:
        if "|--|" in line:
            parts = line.strip().split("|--|")
            paper_id = int(parts[0])
            author_list = parts[1].split(",")
            authors[paper_id] = author_list
    print(f"Loaded {len(authors)} authors.")

Loaded 138499 authors.


In [ ]:
def generate_bert_embeddings(abstracts, authors, tokenizer_bert, model_bert, batch_size=6):

    bert_embeddings = {}
    paper_ids = list(abstracts.keys())

    # Process all papers in batches
    for i in tqdm(range(0, len(paper_ids), batch_size)):
        batch_ids = paper_ids[i:i+batch_size]
        batch_texts = []

        # Prepare batch texts with fallback
        for pid in batch_ids:
            abstract = abstracts.get(pid, "").strip()

            if abstract:
                # Use abstract
                batch_texts.append(abstract)
            else:
                # Fallback to authors
                author_list = authors.get(pid, [])
                if author_list:
                    batch_texts.append(f"Authors: {', '.join(author_list)}")
                else:
                    batch_texts.append("No information available")

        # Generate embeddings for batch
        inputs = tokenizer_bert(batch_texts, padding=True, truncation=True,
                               max_length=512, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model_bert(**inputs)

        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

        # Store embeddings
        for idx, paper_id in enumerate(batch_ids):
            bert_embeddings[paper_id] = embeddings[idx]

    return bert_embeddings

In [ ]:
# Load pre-trained BERT model
print("Loading BERT model...")

# SPECTER
print("Loading SPECTER model...")
tokenizer_specter = AutoTokenizer.from_pretrained('allenai/specter')
model_specter = AutoModel.from_pretrained('allenai/specter')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_specter = model_specter.to(device)
print(f"Device: {device}")

# Generate SPECTER embeddings
print("Generating SPECTER embeddings for abstracts...")
specter_embeddings = generate_bert_embeddings(abstracts, authors, tokenizer_bert=tokenizer_specter , model_bert=model_specter)
print(f"\nGenerated SPECTER embeddings for {len(specter_embeddings)} papers")
np.save('/content/drive/MyDrive/llms/specter_embeddings.npy', specter_embeddings)

# DistilBERT (assuming you want to use this model)
print("Loading DistilBERT model...")
tokenizer_distilbert = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model_distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")
model_distilbert = model_distilbert.to(device)
print(f"Device: {device}")

# Generate DistilBERT embeddings
print("Generating DistilBERT embeddings for abstracts...")
distilbert_embeddings = generate_bert_embeddings(abstracts, authors, tokenizer_bert=tokenizer_distilbert , model_bert=model_distilbert)
print(f"\nGenerated DistilBERT embeddings for {len(distilbert_embeddings)} papers")
np.save('/content/drive/MyDrive/llms/distilbert_embeddings.npy', distilbert_embeddings)

# SciBERT (assuming you want to use this model as well)
print("Loading SciBERT model...")
tokenizer_scibert = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model_scibert = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")
model_scibert = model_scibert.to(device)
print(f"Device: {device}")

# Generate SciBERT embeddings
print("Generating SciBERT embeddings for abstracts...")
scibert_embeddings = generate_bert_embeddings(abstracts, authors, tokenizer_bert=tokenizer_scibert , model_bert=model_scibert)
print(f"\nGenerated SciBERT embeddings for {len(scibert_embeddings)} papers")
np.save('/content/drive/MyDrive/llms/scibert_embeddings.npy', scibert_embeddings)

Loading BERT model...
Loading SPECTER model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

...

Device: cuda
Generating SciBERT embeddings for abstracts...


  0%|          | 27/23084 [00:00<13:59, 27.47it/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

100%|██████████| 23084/23084 [17:17<00:00, 22.25it/s]



Generated SciBERT embeddings for 138499 papers
